In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

In [3]:
from underactuated import PlanarRigidBodyVisualizer
from pydrake.all import (FloatingBaseType,
                         RigidBodyTree,
                         RigidBodyPlant,
                         DiagramBuilder,
                         ZeroOrderHold,
                         SignalLogger,
                         Simulator)

In [4]:
from pympc.geometry.polyhedron import Polyhedron

In [5]:
from utils import pwa_from_RigidBodyPlant, Controller

In [6]:
# linearization points for the angle of the pole
th_min = - np.pi * .3
th_max = np.pi * .3
n_lin = 5
th_step = (th_max - th_min) / (n_lin - 1)
th_lin = [np.array([th]) for th in np.linspace(th_min, th_max, n_lin)]

In [7]:
# overall boundaries for the state
x_min = np.array([[-3.], [th_min-th_step], [-10.], [-10.]])
x_max = np.array([[3.], [th_max+th_step], [10.], [10.]])
X = Polyhedron.from_bounds(x_min, x_max)

In [8]:
# overall boundaries for the input
u_min = np.array([[-100.]])
u_max = np.array([[100.]])
U = Polyhedron.from_bounds(u_min, u_max)

In [9]:
# RigidBodyPlant from urdf
tree = RigidBodyTree("cart_pole.urdf", FloatingBaseType.kFixed)
plant = RigidBodyPlant(tree)

In [10]:
# discretization time
h = .05

In [11]:
# piecewise affine system
pwa = pwa_from_RigidBodyPlant(plant, h, X, U, th_lin, indices=[1])

In [12]:
# drake controller
N = 20
Q = np.eye(4)/100.
R = np.eye(1)/100.
P = Q
X_N = Polyhedron.from_bounds(*[np.zeros((pwa.nx,1))]*2)
# from copy import copy
# X_N = copy(X)
controller = Controller(pwa, N, Q, R, Q, X_N, method='Convex hull, lifted constraints')

In [13]:
# blocks of the diagram
builder = DiagramBuilder()
robot = builder.AddSystem(plant)
controller = builder.AddSystem(controller)
zoh = builder.AddSystem(ZeroOrderHold(.1, 1))

In [14]:
# logger
logger_freq = 30.
logger = SignalLogger(plant.get_num_states())
state_log = builder.AddSystem(logger)
state_log._DeclarePeriodicPublish(1./logger_freq, 0.0)

In [15]:
# connect and build the block diagram
builder.Connect(robot.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), zoh.get_input_port(0))
builder.Connect(zoh.get_output_port(0), robot.get_input_port(0))
builder.Connect(plant.get_output_port(0), state_log.get_input_port(0))
diagram = builder.Build()

In [16]:
# set up sim
simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False)
integrator = simulator.get_mutable_integrator()
integrator.set_fixed_step_mode(True)
integrator.set_maximum_step_size(0.01)

In [17]:
# simulate
x0 = np.array([0., 0., 1., 0.])
state = simulator.get_mutable_context().get_mutable_continuous_state_vector()
state.SetFromVector(x0)
simulator.StepTo(5.)

In [20]:
# visualizer
viz = PlanarRigidBodyVisualizer(tree, xlim=[-1, 1], ylim=[-2.5, 3.])
viz.fig.set_size_inches(10, 5)
ani = viz.animate(state_log, 30, repeat=True)
plt.close(viz.fig)
HTML(ani.to_html5_video())

Spawning PlanarRigidBodyVisualizer for tree with 1 actuators


In [19]:
'''
ISSUES:
    - the ZOH block introduces a delay at the first time step;
'''

'\nISSUES:\n    - the ZOH block introduces a delay at the first time step;\n'